STT(Speech To Text) : 음성 입력 => 텍스트 출력

In [ ]:
import gradio as gr

with gr.Blocks() as demo : 

    # 오디오 파일 경로 반환
    def change_audio(audio_path) : 
        return audio_path
    
    with gr.Row() :
        input_mic = gr.Audio(sources="microphone", type="filepath", label="Input Microphone")   # 마이크로 음성 녹음
        input_file = gr.Audio(sources="upload", type="filepath", label="Input File")            # 녹음된 파일 업로드

        output_text = gr.Textbox(label="Transcription Output")  # 텍스트 박스

        # 마이크로 음성 녹음 완료 시, 녹음된 파일 경로를 output_text에 표출
        input_mic.change(fn=change_audio, inputs=[input_mic], outputs=[output_text])

demo.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [3]:
import gradio as gr

with gr.Blocks() as demo : 

    # 오디오 파일 경로 반환
    def request_stt(audio_path) :
        # 1. Endpoint
        endpoint = "https://eastus.stt.speech.microsoft.com/speech/recognition/conversation/cognitiveservices/v1?language=ko-KR&format=detailed"

        # 2. Header
        headers = {
            "Content-Type" : "audio/wav",
            "Ocp-Apim-Subscription-Key" : os.getenv("STT_OCP_APIM_SUB_KEY")
        }

        # 3. Body : Audio file => binary
        with open(audio_path, "rb") as audio_file : 
            audio_data = audio_file.read()

        body = audio_data

        # request Audio => Text
        response = requests.post(endpoint, headers=headers, data=body)

        if response.status_code != 200 :
            print(f"Http Error {response.status_code} : {response.reason}")
            return None
        
        import json

        # String => Dictionary
        result_json = json.loads(response.text)

        if result_json.get("RecognitionStatus") != "Success" : 
            print("Response Error ", result_json.get("RecognitionStatus"))
            return None
        
        # return Audio => Text
        return result_json.get("DisplayText")

    def change_audio(audio_path) :
        return audio_path
    
    def click_send(audio_path) :
        if audio_path is None : 
            return "마이크를 녹음해주세요."
        
        return request_stt(audio_path)

    with gr.Row() :
        input_mic = gr.Audio(sources="microphone", type="filepath", label="Input Microphone")   # 마이크로 음성 녹음
        input_file = gr.Audio(sources="upload", type="filepath", label="Input File")            # 녹음된 파일 업로드

        file_path_text = gr.Textbox(label="Transcription Output")  # 텍스트 박스

    # 마이크로 음성 녹음 완료 시, 녹음된 파일 경로를 output_text에 표출
    input_mic.change(fn=change_audio, inputs=[input_mic], outputs=[file_path_text])
    
    # 녹음 이후, 전송 버튼 클릭 시, click_send에서 녹음 여부 판단 후, Audio => 텍스트로 변환하여 변환된 텍스트를 output_text에 출력
    send_button = gr.Button("전송")
    output_text = gr.Textbox(label="결과 텍스트")

    send_button.click(fn=click_send, inputs=[input_file], outputs=[output_text])

demo.launch()

import requests
import os
from dotenv import load_dotenv
# .env 환경변수 로드
load_dotenv()

    

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


True